In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Constants
TOOL_MENU_HEIGHT = 50
TOOL_MENU_MARGIN = 150
MENU_WIDTH = 250 + TOOL_MENU_MARGIN
MENU_HEIGHT = 50
RADIUS_INIT = 40
THICKNESS = 4
DETECTION_CONFIDENCE = 0.6
TRACKING_CONFIDENCE = 0.6

# Tool selection function
def get_tool(x):
    if x < 50 + TOOL_MENU_MARGIN:
        return "line"
    elif x < 100 + TOOL_MENU_MARGIN:
        return "rectangle"
    elif x < 150 + TOOL_MENU_MARGIN:
        return "draw"
    elif x < 200 + TOOL_MENU_MARGIN:
        return "circle"
    else:
        return "erase"

# Check if index finger is raised
def index_raised(y_index, y_middle):
    return (y_middle - y_index) > 40

# Initialize MediaPipe hands model
mp_hands = mp.solutions.hands
hand_landmark = mp_hands.Hands(min_detection_confidence=DETECTION_CONFIDENCE, min_tracking_confidence=TRACKING_CONFIDENCE, max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Load tools image
tools = cv2.imread("tools.png")

# Check if the tools image was loaded properly
if tools is None:
    print("Error: tools.png image not found.")
    # Create a dummy tools image as a fallback
    tools = np.zeros((MENU_HEIGHT, MENU_WIDTH - TOOL_MENU_MARGIN, 3), dtype='uint8')
    cv2.putText(tools, 'Tools', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Create a mask for drawing
mask = np.ones((480, 640), dtype='uint8') * 255

# Initialize video capture
cap = cv2.VideoCapture(0)
curr_tool = "select tool"
time_init = True
radius = RADIUS_INIT
var_inits = False
prev_x, prev_y = 0, 0

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    result = hand_landmark.process(rgb_frame)
    
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            x, y = int(hand_landmarks.landmark[8].x * 640), int(hand_landmarks.landmark[8].y * 480)

            if TOOL_MENU_MARGIN < x < MENU_WIDTH and y < MENU_HEIGHT:
                if time_init:
                    start_time = time.time()
                    time_init = False
                
                cv2.circle(frame, (x, y), radius, (0, 255, 255), 2)
                radius -= 1
                
                if time.time() - start_time > 0.8:
                    curr_tool = get_tool(x)
                    print(f"Your current tool set to: {curr_tool}")
                    time_init = True
                    radius = RADIUS_INIT
            else:
                time_init = True
                radius = RADIUS_INIT

            if curr_tool == "draw":
                x_index, y_index = int(hand_landmarks.landmark[12].x * 640), int(hand_landmarks.landmark[12].y * 480)
                y_middle = int(hand_landmarks.landmark[9].y * 480)

                if index_raised(y_index, y_middle):
                    cv2.line(mask, (prev_x, prev_y), (x, y), 0, THICKNESS)
                    prev_x, prev_y = x, y
                else:
                    prev_x, prev_y = x, y

            elif curr_tool == "line":
                x_index, y_index = int(hand_landmarks.landmark[12].x * 640), int(hand_landmarks.landmark[12].y * 480)
                y_middle = int(hand_landmarks.landmark[9].y * 480)

                if index_raised(y_index, y_middle):
                    if not var_inits:
                        start_x, start_y = x, y
                        var_inits = True

                    cv2.line(frame, (start_x, start_y), (x, y), (50, 152, 255), THICKNESS)
                else:
                    if var_inits:
                        cv2.line(mask, (start_x, start_y), (x, y), 0, THICKNESS)
                        var_inits = False

            elif curr_tool == "rectangle":
                x_index, y_index = int(hand_landmarks.landmark[12].x * 640), int(hand_landmarks.landmark[12].y * 480)
                y_middle = int(hand_landmarks.landmark[9].y * 480)

                if index_raised(y_index, y_middle):
                    if not var_inits:
                        start_x, start_y = x, y
                        var_inits = True

                    cv2.rectangle(frame, (start_x, start_y), (x, y), (0, 255, 255), THICKNESS)
                else:
                    if var_inits:
                        cv2.rectangle(mask, (start_x, start_y), (x, y), 0, THICKNESS)
                        var_inits = False

            elif curr_tool == "circle":
                x_index, y_index = int(hand_landmarks.landmark[12].x * 640), int(hand_landmarks.landmark[12].y * 480)
                y_middle = int(hand_landmarks.landmark[9].y * 480)

                if index_raised(y_index, y_middle):
                    if not var_inits:
                        start_x, start_y = x, y
                        var_inits = True

                    radius = int(((start_x - x) ** 2 + (start_y - y) ** 2) ** 0.5)
                    cv2.circle(frame, (start_x, start_y), radius, (255, 255, 0), THICKNESS)
                else:
                    if var_inits:
                        radius = int(((start_x - x) ** 2 + (start_y - y) ** 2) ** 0.5)
                        cv2.circle(mask, (start_x, start_y), radius, 0, THICKNESS)
                        var_inits = False

            elif curr_tool == "erase":
                x_index, y_index = int(hand_landmarks.landmark[12].x * 640), int(hand_landmarks.landmark[12].y * 480)
                y_middle = int(hand_landmarks.landmark[9].y * 480)

                if index_raised(y_index, y_middle):
                    cv2.circle(frame, (x, y), 30, (0, 0, 0), -1)
                    cv2.circle(mask, (x, y), 30, 255, -1)

    # Combine the original frame with the mask
    frame[:, :, 1] = cv2.bitwise_and(frame, frame, mask=mask)[:, :, 1]
    frame[:, :, 2] = cv2.bitwise_and(frame, frame, mask=mask)[:, :, 2]

    # Overlay the tool menu
    frame[:MENU_HEIGHT, TOOL_MENU_MARGIN:MENU_WIDTH] = cv2.addWeighted(tools, 0.7, frame[:MENU_HEIGHT, TOOL_MENU_MARGIN:MENU_WIDTH], 0.3, 0)

    # Display the current tool text
    cv2.putText(frame, curr_tool, (270 + TOOL_MENU_MARGIN, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the frame
    cv2.imshow("Paint App", frame)

    # Break loop on 'Esc' key press
    if cv2.waitKey(1) == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Constants
TOOL_MENU_HEIGHT = 50
TOOL_MENU_MARGIN = 150
MENU_WIDTH = 250 + TOOL_MENU_MARGIN
MENU_HEIGHT = 50
RADIUS_INIT = 40
THICKNESS = 4
DETECTION_CONFIDENCE = 0.6
TRACKING_CONFIDENCE = 0.6

# Tool selection function
def get_tool(x):
    if x < 125 + TOOL_MENU_MARGIN:
        return "draw"
    else:
        return "erase"

# Check if index finger is raised
def index_raised(y_index, y_middle):
    return (y_middle - y_index) > 40

# Initialize MediaPipe hands model
mp_hands = mp.solutions.hands
hand_landmark = mp_hands.Hands(min_detection_confidence=DETECTION_CONFIDENCE, min_tracking_confidence=TRACKING_CONFIDENCE, max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Load tools image
tools = cv2.imread("tools.png")

# Check if the tools image was loaded properly
if tools is None:
    print("Error: tools.png image not found.")
    # Create a dummy tools image as a fallback
    tools = np.zeros((MENU_HEIGHT, MENU_WIDTH - TOOL_MENU_MARGIN, 3), dtype='uint8')
    cv2.putText(tools, 'Draw | Erase', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Create a mask for drawing
mask = np.ones((480, 640), dtype='uint8') * 255

# Initialize video capture
cap = cv2.VideoCapture(0)
curr_tool = "select tool"
time_init = True
radius = RADIUS_INIT
prev_x, prev_y = 0, 0

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    result = hand_landmark.process(rgb_frame)
    
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            x, y = int(hand_landmarks.landmark[8].x * 640), int(hand_landmarks.landmark[8].y * 480)

            if TOOL_MENU_MARGIN < x < MENU_WIDTH and y < MENU_HEIGHT:
                if time_init:
                    start_time = time.time()
                    time_init = False
                
                cv2.circle(frame, (x, y), radius, (0, 255, 255), 2)
                radius -= 1
                
                if time.time() - start_time > 0.8:
                    curr_tool = get_tool(x)
                    print(f"Your current tool set to: {curr_tool}")
                    time_init = True
                    radius = RADIUS_INIT
            else:
                time_init = True
                radius = RADIUS_INIT

            if curr_tool == "draw":
                x_index, y_index = int(hand_landmarks.landmark[12].x * 640), int(hand_landmarks.landmark[12].y * 480)
                y_middle = int(hand_landmarks.landmark[9].y * 480)

                if index_raised(y_index, y_middle):
                    cv2.line(mask, (prev_x, prev_y), (x, y), 0, THICKNESS)
                    prev_x, prev_y = x, y
                else:
                    prev_x, prev_y = x, y

            elif curr_tool == "erase":
                x_index, y_index = int(hand_landmarks.landmark[12].x * 640), int(hand_landmarks.landmark[12].y * 480)
                y_middle = int(hand_landmarks.landmark[9].y * 480)

                if index_raised(y_index, y_middle):
                    cv2.circle(frame, (x, y), 30, (0, 0, 0), -1)
                    cv2.circle(mask, (x, y), 30, 255, -1)

    # Combine the original frame with the mask
    frame[:, :, 1] = cv2.bitwise_and(frame, frame, mask=mask)[:, :, 1]
    frame[:, :, 2] = cv2.bitwise_and(frame, frame, mask=mask)[:, :, 2]

    # Overlay the tool menu
    frame[:MENU_HEIGHT, TOOL_MENU_MARGIN:MENU_WIDTH] = cv2.addWeighted(tools, 0.7, frame[:MENU_HEIGHT, TOOL_MENU_MARGIN:MENU_WIDTH], 0.3, 0)

    # Display the current tool text
    cv2.putText(frame, curr_tool, (270 + TOOL_MENU_MARGIN, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the frame
    cv2.imshow("Paint App", frame)

    # Break loop on 'Esc' key press
    if cv2.waitKey(1) == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
